In [4]:
import pandas as pd
from datetime import datetime


# Load Data

In [5]:
nba       = pd.read_csv('../data/nba/nba_salaries.csv')
insurance = pd.read_csv('../data/insurance/insurance.csv')
airline   = pd.read_csv('../data/airline/train.csv')
airbnb    = pd.read_csv('../data/airbnb/listings.csv')

# Basic Data Wrangling


Function to handle the following issues:

1. Empty columns;
2. Categorical columns where the majority of values are unique
3. Columns with only one unique value

In [8]:
def basic_wrangling(df: pd.DataFrame, columns : list = None, 
                    proportion_nan_thresh : float = 0.95, 
                    proprotion_unique_thresh : float = 0.95):
    '''
    Function to execute some basic data wrangling in pandas dataframe. 
    The following operations will be executed:
        
        1. Remove empty or almost empty columns 
        2. Remove categorical columns with more than proprotion_unique_thresh 
            unique values
        3. Remove columns composed by a single value

    Args:
        df - pandas Dataframe
        columns - list of columns to be wrngled. If None, operations will 
                  be applied to all columns in dataframe
        proportion_nan_thresh - If the proportion of NAN values in a column 
                    is greater than proportion_nan_thresh the column will be 
                    removed. Default = 0.95
        proportion_unique_thresh - If the proportion of unique values in a 
                    categorical column is greater than proportion_unique_thresh 
                    the column will be removed. Default = 0.95

    Returns:
        df - wrangled pandas dataframe
    '''

    df = df.copy()

    if columns is None:
        columns = df.columns

    n_rows = len(df) #number of rows

    for feature in columns:
        if feature in df.columns:
            #remove empty or almost empy columns
            num_empty_values = df[feature].isna().sum()
            proportion_nan_values = num_empty_values / n_rows
            num_unique = df[feature].nunique()
            proportion_unique_values = num_unique / n_rows
            data_type = df[feature].dtype
            if proportion_nan_values > proportion_nan_thresh:
                df = df.drop(columns = feature)
                print(
                f'Feature {feature} removed. '
                f'{100 * proportion_nan_values:.2f}% of missing values.'
                )

            #remove features with only one unique value
            elif num_unique == 1:
                df = df.drop(columns = feature)
                print(f'Feature {feature} removed. ' 
                      'There is only one unique value in the column.')
                
            #Remove categorical columns with more than proprotion_unique_thresh 
            #unique values
            elif (proportion_unique_values > proprotion_unique_thresh 
                  and not pd.api.types.is_numeric_dtype(df[feature])):
                df= df.drop(columns = feature)
                print(f'Feature {feature} removed. '
                      f'The proportion of unique values in the feature of '
                      f'type  {data_type} is ' 
                      f'{100 * proportion_unique_values:.2f}%')

        else:
            print(f'The feature {feature} is not in the dataframe.')

    

In [ ]:
def parse_and_format_dates(date_string: str, standard_format: str = "%Y-%m-%d", return_type: str = 'string'):
    '''
    Função que converte uma string com data para um objeto datetime. A função irá testar se a string passada
    como input atende um dos critérios aceitos para datas, caso contrário retorna None. Pode ser escolhi o tipo 
    objeto que será retornado, ou uma string ou datetime. Caso seja retornado uma string ela pode ser for formatada 
    para um formato padrão especificado em standard_format.

    Os formatos aceitos para datas são os seguintes:

                "%Y-%m-%d",
                "%d-%m-%Y",
                "%m/%d/%Y",
                "%d %b %Y",
                "%B %d, %Y"

    Args:
        date_string: data no formato string
        standart_format: formato de dados que será retornado
        return_type: tipo de objeto que serṕa retornado, datetime ou string.

    Retuns:
        Datetime object bo formato definido por standard_format
    '''

    #lista com formatos de datas
    formats = [ "%Y-%m-%d",
                "%d-%m-%Y",
                "%m/%d/%Y",
                "%d %b %Y",
                "%B %d, %Y"]
    
    #@testar se input é string, caso negativo retorna Nan
    if not isinstance(date_string, str):
        return pd.NA
    
    if return_type != 'string' or return_type != 'datetime':
        print (f'{return_type} não reconhecifo. Os formatos aceitos são string e datetime. Será usado string como padrão')
        return_type = 'string'
        
    
    for expected_format in formats:
        try:
            #converte para datetime usando o formato especificado em expected_format
            parsed_date = datetime.strptime(date_string, expected_format) 
            if return_type == 'datetime':
                return parsed_date #retorna objeto datetime
            else:
                return parsed_date.strftime(standard_format) #retorna string 
        except:
            continue
    
    return None
    


In [ ]:
def create_new_date_columns(df, features_list: list, reference_date: str = None, 
                            calcular_diferenca: bool = True):
    '''
    Função para converter as colunas especificadas para o formato datetime. Para cada coluna com data, será criada uma nova com ano,
    dia do mês, mês e dia da semana. Além disso, se for passada uma reference_date será criada uma coluna coma diferença entre a 
    reference_date e as datas na feature e uma coluna com a mesma diferença em dias.

    Args:
        df: pandas dataframe
        features_list: lista com as features que serão analisadas
        reference_date: data que será usada para calcular a diferença com relação a feature analisada. Se reference_date = None, 
                        será usado o dia de hoje como reference_date. A fórmula para o cálcula da diferença é a seguinte:
                        reference_date - df[feature]
        date_format - formato esperado da data
        calcular_diferenca - define se será calculada ou não a diferença entre as datas.

    Returns
        df após as transformações aplicadas as colunas especificadas
    '''


    #lista com formatos de datas
    formats = [ "%Y-%m-%d",
                "%d-%m-%Y",
                "%m/%d/%Y",
                "%d %b %Y",
                "%B %d, %Y"]

    for feature in features_list:
        if feature in df.columns:
            #Converter a coluna para datetime
            df[feature] = pd.to_datetime(df[feature])
            
            # Criar três novas colunas, uma com ano, outra com mês, a terceira com o dia do mês e por fim uma com o dia da semana
            df[f'{feature}_year'] = df[feature].dt.year
            df[f'{feature}_month'] = df[feature].dt.month
            df[f'{feature}_day'] = df[feature].dt.day
            df[f'{feature}_weekday'] = df[feature].dt.day_name()

            if calcular_diferenca:
                
                if reference_date is None:
                    parsed_reference_date = datetime.today()
                else:
                    parsed_reference_date = None
                    for expected_format in formats:
                        try:
                            parsed_reference_date = datetime.strptime(reference_date, expected_format)
                            break
                        except:
                            continue
                    if not 'parsed_reference_date':
                        print('Reference date não está em nennhum dos formatos aceito. \n' \
                             f'Será usada como reference_date o dia de hoje: {datetime.today()}')
                        parsed_reference_date = datetime.today()
            
                # diferença entre a reference data e a data na feature
                df[f'actual_date - {feature}'] = reference_date - df[feature]
                #diferença entre a reference_date e a data na feature em dias
                df[f'actual_date - {feature} in days'] = (reference_date - df[feature]).dt.days

        else:
            print(f'A feature {feature} não está no dataframe')

    return df
